In [2]:
# !pip install python-binance
from datetime import datetime
import pandas as pd

In [4]:
api_key = 'V0RnTw2vdjBcizJARYI8qWCT99Vej1DoDBAayNodNRLPKM6Nho87m2LN83JbL9o9'
api_secret = 'e5PIk5tkns5hSjE6qsc4JKSffVWO6VTbQuOY0xjgEBraKo6shnK3iC9rV2z1S2YI'

from datetime import datetime
import pandas as pd

COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 
                   'tb_base_av', 'tb_quote_av', 'ignore']

from binance.client import Client
client = Client(api_key=api_key, api_secret=api_secret)

def get_data(start_date, end_date, symbol):
    data = client.futures_historical_klines(
            symbol=symbol,
            interval='1m',
            start_str=start_date,
            end_str=end_date
        )
    
    # 전처리
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['open_time'] = df['open_time'].apply(lambda x: datetime.fromtimestamp(x // 1000))
    df = df.drop(columns=['close_time', 'ignore'])
    df['symbol'] = symbol
    df.loc[:, 'open':'tb_quote_av'] = df.loc[:, 'open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    return df

start_date = '2024-01-14'
end_date = '2024-01-15'
symbol = ['BTCUSDT']
for i in symbol:
    df = get_data(start_date=start_date, end_date=end_date, symbol=i)
    df.to_csv(f'jh_binance_{i}.csv', index=False)